In [25]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pathlib import Path
# Step 1: Create a session and load the page
url1 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals'
url2 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals'
url3 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals'
url_list = [url1,url2,url3,url4]
#url_list =[url +'&SeasonType=Playoffs' for url in url_list]
url_list =[url +'&SeasonType=Regular+Season'for url in url_list]

def get_tables(url_list):
    data = []
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    for url in url_list:
        
        driver.get(url)
        print(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())

        
        df= dfs[-1]
        #print(df)
        drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        df.columns = df.columns.droplevel()
        df = df.drop(columns = drop)
        data.append(df)
    driver.close()
    return data
#url_list = [url1]#
def multiyear_shooting(url_list,team_round=0,playoffs = True):
    df_list = []
    start_year = 2023
    for i in range(start_year,2024):
        year = i+1
        season = '&Season='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        if team_round!=0:
            year_url = [url + '&PORound='+str(team_round) for url in year_url]
        frames = get_tables(year_url)

 
        path = str(i+1)+'/playoff_shooting/round'+str(team_round)+'/'
        if playoffs == False:
            path = str(i+1)+'/team_shooting/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['very_tight','tight','open','wide_open']
        
        
        for k in range(len(terms)):
            
            frames[k]['shot_coverage'] = terms[k]
            if playoffs == True:
                frames[k]['round'] = team_round
            frames[k]['year'] = year
        df = pd.concat(frames)
        df.to_csv(path+'team_shooting.csv',index = False)
        df_list.append(df)
    new_df = pd.concat(df_list)
    new_df['TEAMNAME'] = new_df['TEAM']
    df = pd.read_csv('team_shooting.csv')
    df = df[df.year <=start_year]
    names = dict(zip(df.TEAMNAME,df.TEAM))
    names['Los Angeles Clippers'] = 'LAC'
    names['Charlotte Bobcats'] = 'CHA'

    #names
    print(names)
    final_df = pd.concat([df,new_df])
    final_df.replace({'TEAM':names},inplace=True)
    final_df.loc[final_df['TEAMNAME'] =='Los Angeles Clippers', 'TEAM'] = 'LAC'
    final_df.loc[final_df['TEAMNAME'] =='Charlotte Bobcats', 'TEAM'] = 'CHA'

    return final_df

df = multiyear_shooting(url_list,playoffs=False)
print(df)

https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=0-2+Feet+-+Very+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=2-4+Feet+-+Tight&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=4-6+Feet+-+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
https://www.nba.com/stats/teams/shots-closest-defender?CloseDefDistRange=6%2B+Feet+-+Wide+Open&PerMode=Totals&SeasonType=Regular+Season&Season=2023-24
{'Portland Trail Blazers': 'POR', 'New York Knicks': 'NYK', 'Dallas Mavericks': 'DAL', 'Los Angeles Lakers': 'LAL', 'Denver Nuggets': 'DEN', 'Cleveland Cavaliers': 'CLE', 'Orlando Magic': 'ORL', 'Los Angeles Clippers': 'LAC', 'Washington Wizards': 'WAS', 'Boston Celtics': 'BOS', 'Golden State Warriors': 'GSW', 'Sacramento Kings': 'SAC', 'Minnesota Timberwolves': 'MIN', 'Phoenix Suns': 'PHX', 'Oklahoma Ci

In [27]:
df.to_csv('team_shooting.csv',index = False)

In [26]:
df.head(40)

,TEAM,GP,G,Freq%,FGM,FGA,FG%,eFG%,2FG Freq%,2FGM,2FGA,2FG%,3FG Freq%,3PM,3PA,3P%,shot_coverage,year,TEAMNAME
0,POR,82,82,28.4,862,1942,44.4,51.5,17.0,587,1161,50.6,11.4,275,781,35.2,open,2014,Portland Trail Blazers
1,NYK,82,82,29.1,826,1877,44.0,50.7,18.0,576,1159,49.7,11.1,250,718,34.8,open,2014,New York Knicks
2,DAL,82,82,28.4,882,1864,47.3,54.3,17.8,621,1168,53.2,10.6,261,696,37.5,open,2014,Dallas Mavericks
3,LAL,82,82,28.0,786,1844,42.6,49.8,16.7,521,1103,47.2,11.2,265,741,35.8,open,2014,Los Angeles Lakers
4,DEN,82,82,27.7,794,1823,43.6,51.0,16.4,524,1079,48.6,11.3,270,744,36.3,open,2014,Denver Nuggets
5,CLE,82,82,28.0,793,1820,43.6,49.4,18.8,580,1223,47.4,9.2,213,597,35.7,open,2014,Cleveland Cavaliers
6,ORL,82,82,28.7,797,1819,43.8,50.1,19.1,570,1212,47.0,9.6,227,607,37.4,open,2014,Orlando Magic
7,LAC,82,82,27.9,856,1809,47.3,53.8,17.8,622,1155,53.9,10.1,234,654,35.8,open,2014,Los Angeles Clippers
8,WAS,81,81,27.1,767,1765,43.5,48.9,18.8,574,1228,46.7,8.2,193,537,35.9,open,2014,Washington Wizards
9,BOS,82,82,27.1,772,1761,43.8,49.6,18.2,569,1186,48.0,8.8,203,575,35.3,open,2014,Boston Celtics
